<a href="https://colab.research.google.com/github/aekanun2020/2025-AdvancedRAG/blob/main/SENT_v4_Hands_on_1_Splitter_and_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ติดตั้ง LlamaIndex และ dependencies
!pip install llama-index -q
!pip install llama-index-embeddings-huggingface -q

# Import modules
import os
import urllib.request
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.node_parser import MarkdownNodeParser, SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# ฟังก์ชันดาวน์โหลด corpus
def download_corpus():
    import urllib.request
    import os
    os.makedirs('./corpus_input', exist_ok=True)
    urls = [
        ("https://storage.googleapis.com/llm-course/md/1.md", "./corpus_input/1.md"),
        ("https://storage.googleapis.com/llm-course/md/2.md", "./corpus_input/2.md"),
        ("https://storage.googleapis.com/llm-course/md/44.md", "./corpus_input/44.md"),
        ("https://storage.googleapis.com/llm-course/md/5555.md", "./corpus_input/5555.md")
    ]
    for url, path in urls:
        if not os.path.exists(path):
            print(f"Downloading {url} to {path}")
            try:
                urllib.request.urlretrieve(url, path)
            except Exception as e:
                print(f"Failed to download {url}: {e}")

# ดาวน์โหลด corpus
print("กำลังดาวน์โหลดไฟล์...")
download_corpus()

# โหลดเอกสาร Markdown จากไดเรกทอรี
reader = SimpleDirectoryReader(
    input_dir="./corpus_input",
    recursive=True,
    required_exts=[".md", ".markdown"]
)
documents = reader.load_data()
print(f"โหลดเอกสาร {len(documents)} ไฟล์สำเร็จ")

##############################################
# ส่วนที่ 1: การเปรียบเทียบระหว่าง Parser และ Splitter
##############################################

print("\n=== การเปรียบเทียบระหว่าง MarkdownNodeParser กับ SentenceSplitter ===\n")

# สร้าง parsers และ splitters
md_parser = MarkdownNodeParser()
sentence_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=50)

# เปรียบเทียบผลลัพธ์สำหรับแต่ละไฟล์
for doc in documents:
    filename = doc.metadata.get('file_name', 'Unknown')
    print(f"\n>> ไฟล์: {filename}")

    # ทดสอบ MarkdownNodeParser
    parser_nodes = md_parser.get_nodes_from_documents([doc])

    # ทดสอบ SentenceSplitter
    splitter_nodes = sentence_splitter.get_nodes_from_documents([doc])

    # เปรียบเทียบจำนวน nodes
    print(f"MarkdownNodeParser: {len(parser_nodes)} nodes")
    print(f"SentenceSplitter: {len(splitter_nodes)} nodes")

    # แสดงตัวอย่าง nodes จาก Parser
    print(f"\nตัวอย่าง nodes จาก MarkdownNodeParser (3 ตัวอย่างแรก):")
    for i, node in enumerate(parser_nodes[:3]):
        print(f"\n  Node {i+1}:")
        print(f"  Header Path: {node.metadata.get('header_path', 'No header path')}")
        print(f"  Text: {node.text}")

    # แสดงตัวอย่าง nodes จาก Splitter
    print(f"\nตัวอย่าง nodes จาก SentenceSplitter (3 ตัวอย่างแรก):")
    for i, node in enumerate(splitter_nodes[:3]):
        print(f"\n  Node {i+1}:")
        print(f"  Text: {node.text}")

    # แสดงตัวอย่างเฉพาะสำหรับเนื้อหาที่มีหัวข้อชัดเจน (สำหรับไฟล์ 1.md)
    if filename == "1.md":
        print("\n=== ตัวอย่างเฉพาะ: การเปรียบเทียบเนื้อหาเกี่ยวกับ 'สาเหตุของโรคหัดเยอรมัน' ===")

        # หา nodes ที่เกี่ยวกับ "สาเหตุของโรคหัดเยอรมัน" จาก MarkdownNodeParser โดยค้นหาจาก text
        cause_parser_nodes = [node for node in parser_nodes if "สาเหตุของโรคหัดเยอรมัน" in node.text]
        if cause_parser_nodes:
            print("\nMarkdownNodeParser: เนื้อหาสาเหตุของโรคหัดเยอรมันอยู่ใน node เดียว")
            node = cause_parser_nodes[0]
            print(f"  Header Path: {node.metadata.get('header_path', 'No header path')}")
            print(f"  Text: {node.text}")

        # หา nodes ที่เกี่ยวกับ "สาเหตุของโรคหัดเยอรมัน" จาก SentenceSplitter (อาจกระจายอยู่หลาย nodes)
        cause_splitter_nodes = [node for node in splitter_nodes if "สาเหตุของโรคหัดเยอรมัน" in node.text]
        print(f"\nSentenceSplitter: พบ {len(cause_splitter_nodes)} nodes ที่มีคำว่า 'สาเหตุของโรคหัดเยอรมัน'")
        for i, node in enumerate(cause_splitter_nodes[:2]):  # แสดงเพียง 2 nodes
            print(f"\n  Node {i+1}:")
            print(f"  Text: {node.text}")

# สรุปข้อแตกต่างระหว่าง Parser และ Splitter
print("""
=== สรุปข้อแตกต่างระหว่าง MarkdownNodeParser และ SentenceSplitter ===

1. การแบ่ง nodes:
   - MarkdownNodeParser: แบ่งตามโครงสร้างของ Markdown (ตามหัวข้อ #, ##, ###) รักษาความสัมพันธ์ของเนื้อหาไว้
   - SentenceSplitter: แบ่งตามขนาดที่กำหนด (chunk_size) โดยพยายามรักษาขอบเขตของประโยค ไม่สนใจโครงสร้าง Markdown

2. Metadata:
   - MarkdownNodeParser: เก็บข้อมูลเกี่ยวกับพาธของหัวข้อ (header_path) ซึ่งแสดงลำดับชั้นของหัวข้อ
   - SentenceSplitter: ไม่มีข้อมูลเกี่ยวกับโครงสร้างของเอกสาร ทำให้อาจสูญเสียบริบทของหัวข้อ

3. จำนวน Nodes:
   - MarkdownNodeParser: น้อยกว่า เพราะแบ่งตามหัวข้อเท่านั้น
   - SentenceSplitter: มากกว่า เพราะแบ่งตามขนาดที่กำหนด

4. ลักษณะการทำงาน:
   - MarkdownNodeParser: เน้นการรักษาโครงสร้างและบริบทของหัวข้อ
   - SentenceSplitter: เน้นการควบคุมขนาดของ chunks ให้มีความสม่ำเสมอ

5. การกำหนดค่า:
   - MarkdownNodeParser: ไม่ต้องกำหนดค่า chunk_size หรือ chunk_overlap
   - SentenceSplitter: ต้องกำหนดค่า chunk_size และ chunk_overlap เพื่อควบคุมขนาด node
""")

##############################################
# ส่วนที่ 2: ขั้นตอนการ Ingest จนถึงการสร้าง Indexes
##############################################

print("\n=== ขั้นตอนการ Ingest จนถึงการสร้าง Indexes ===")

# 1. สร้าง Indexes จากแต่ละ parser
print("\n1. การสร้าง Indexes จากแต่ละ parser")

# สร้าง nodes จาก MarkdownNodeParser
md_nodes = md_parser.get_nodes_from_documents(documents)
print(f"  MarkdownNodeParser: สร้าง {len(md_nodes)} nodes")

# ตั้งค่า embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# สร้าง index สำหรับ MarkdownNodeParser
md_index = VectorStoreIndex(nodes=md_nodes)

# สร้าง nodes จาก SentenceSplitter
sentence_nodes = sentence_splitter.get_nodes_from_documents(documents)
print(f"  SentenceSplitter: สร้าง {len(sentence_nodes)} nodes")

# สร้าง index สำหรับ SentenceSplitter
sentence_index = VectorStoreIndex(nodes=sentence_nodes)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.6/263.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

  SentenceSplitter: สร้าง 182 nodes
